# Planning-Lab Lesson 2: Solving Probabilistic Markov Decision Processes

In this second session, we will work on solving problems modelled as goal-based Markov decision processes (also known as Stochastic Shortest Path).

## The environment
The environment used is the same one introduced during theoretical lectures, with some variations:

<img src="images/env.png" width="600">

#### The agent starts in state **d1** and has to reach state **d4**. Some transitions from one state to another have a stochastic outcome (e.g. going from d2 to d3 has a 20% chance of ending in d5) and different costs (i.e. negative rewards, the ones in purple in the figure).

In [10]:
import os, sys, random
# Add the tools directory to the path
tools_path = os.path.abspath(os.path.join(os.getcwd(), '../tools'))
if tools_path not in sys.path:
    sys.path.insert(0, tools_path)

# Add the utils directory to the path
utils_path = os.path.join(tools_path, 'utils')
if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from ai_lab_functions import UCT_log
import gym
from envs import *
from timeit import default_timer as timer
import numpy as np
import random
import math

### Environment Properties 

Here you have some useful functions you can use in the environment:
- **Pr(s' | s, a)**: returns the probability of ending in state **s'** starting from **s** and performing action **a**;
- **$\gamma$(s, a)**: returns all the states in which the agent could end performing action **a** in **s**;
- **Applicable(s)**: returns the actions that can be executed from state **s**;
- **get_cost(s, a, s')**: return the cost (a negative reward) of performing action **a** to go from state **s** to state **s'**.

#### Code Hints:

In [11]:
env = gym.make("ProbabilisticPlanningDomain-v0")

print("Number of actions: ", env.action_space.n)
print("Actions: ", env.actions)
print("States: ", env.states)
print("Probability from d1 to d2 with action m12:", env.Pr[('d1', 'm12', 'd2')])
print("Actions that can be performed in state d2:", env.Applicable('d2'))
print("States in which action m23 performed in state d2 could end:", env.gamma('d2', 'm23'))
print("Cost (already converted into a negative reward) for performing m12 from d1 to d2:", env.get_cost('d1', 'm12', 'd2'))

Number of actions:  11
Actions:  ['m12', 'm14', 'm21', 'm23', 'm32', 'm34', 'm41', 'm43', 'm45', 'm52', 'm54']
States:  ['d1', 'd2', 'd3', 'd4', 'd5']
Probability from d1 to d2 with action m12: 1.0
Actions that can be performed in state d2: ['m23', 'm21']
States in which action m23 performed in state d2 could end: ['d3', 'd5']
Cost (already converted into a negative reward) for performing m12 from d1 to d2: -100


## Assignment 1: Policy Iteration Algorithm

Your first assignment is to implement the Policy Iteration algorithm on the environment presented above. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state. 

<img src="images/policy-iteration.png" width="600">

For the *policy evaluation step*, it is necessary to implement this function:

<img src="images/policy-evaluation.png" width="500">

**The following function has to be implemented:**

In [12]:
def policy_iteration(environment, maxiters=200, maxviters=50, delta=1e-3):
    i = 1
    # Initialize a safe policy
    pi = environment.init_safe_policy()
    # Initialize Value Function
    V = {s: 0 for s in environment.states}
    
    while i <= maxiters:
        print("iteration", i)
        i += 1
        
        # 1) Policy Evaluation
        for j in range(maxviters):
            max_diff = 0
            for state in environment.states:
                # Value of goal state is always 0
                if state == environment.goal_state:
                    V[state] = 0
                    continue
                
                action = pi.get(state)
                # If no action is defined (or terminal), skip
                if action is None:
                    continue
                
                # Bellman Expectation Equation for V
                # V(s) = sum_{s'} P(s'|s,a) * [ Cost(s,a,s') + V(s') ]
                val = 0
                # Get all possible next states from this state-action pair
                possible_next_states = environment.gamma(state, action)
                
                for next_state in possible_next_states:
                    prob = environment.Pr.get((state, action, next_state), 0)
                    cost = environment.get_cost(state, action, next_state)
                    val += prob * (cost + V[next_state])
                
                max_diff = max(max_diff, abs(V[state] - val))
                V[state] = val
            
            # Check for convergence of Value Evaluation
            if max_diff < delta:
                break
        
        # 2) Policy Improvement
        unchanged = True  
        for state in environment.states:
            if state == environment.goal_state:
                continue
            
            old_action = pi.get(state)
            
            best_action = old_action
            best_value = -float('inf') 
            
            # Argmax over actions
            applicable_actions = environment.Applicable(state)
            
            for action in applicable_actions:
                q_val = 0
                possible_next_states = environment.gamma(state, action)
                
                for next_state in possible_next_states:
                    prob = environment.Pr.get((state, action, next_state), 0)
                    cost = environment.get_cost(state, action, next_state)
                    q_val += prob * (cost + V[next_state])
                
                # We strictly maximize the value (negative cost)
                if q_val > best_value:
                    best_value = q_val
                    best_action = action
            
            if best_action != old_action:
                pi[state] = best_action
                unchanged = False
            
        if unchanged: 
            print(f"STOP [unchanged = {unchanged}]")
            break       
    
    return pi

**The following code executes Value Iteration and prints the resulting policy**

In [13]:
env = gym.make("ProbabilisticPlanningDomain-v0")
env.reset()

t = timer()
print("\nINITIAL SAFE POLICY: \n{}".format(env.init_safe_policy()))

policy = policy_iteration(env)

print("\nEXECUTION TIME: {}".format(round(timer() - t, 4)))
print(policy)
print(f"Goal State is: {env.goal_state}")


INITIAL SAFE POLICY: 
{'d1': 'm14', 'd2': 'm23', 'd3': 'm34', 'd4': None, 'd5': 'm54'}
iteration 1
iteration 2
STOP [unchanged = True]

EXECUTION TIME: 0.0008
{'d1': 'm12', 'd2': 'm23', 'd3': 'm34', 'd4': None, 'd5': 'm54'}
Goal State is: d4


# Upper Confidence Trees (UCT) Algorithm

Your second assignment is to implement parts of the UCT algorithm on LavaFloor. This time, the solution returned by your algorithm will be the preferred action to accomplish in a given state, not a full policy. You can perform some preliminary tests on the previous environment and then test your algorithm on the new environment explained below.

## Description
The **Upper Confidence Trees (UCT)** algorithm is a sampling-based method that balances *exploration* and *exploitation* when searching through state-action spaces.

<img src="images/uct_lookahead.png" width="400">

The main **UCT_Lookahead Function** iteratively invokes rollouts until the termination condition is met. After the process, it determines the best action based on the Q-values. 
This implementation applies UCT with a recursive rollout process to evaluate and select the best action in a given state based on accumulated Q-values and visit counts. It operates under a depth-limited search and terminates based on the number of simulations performed.

<img src="images/rollout.png" width="600">

The recursive **UCT Rollout Function** selects actions based on the exploration-exploitation tradeoff to simulate transitions to new states and uses the accumulated cost to update the Q-values for each state-action pair.
The action selection strategy is implemented using a UCT-based formula:

<img src="images/select-UCT-max.png" width="600">

   where:
   - Q(s, a): Action value for state-action pair (s, a);
   - N(s): Total visit count for state s;
   - N(s, a): Visit count for state-action pair (s, a);
   - C: Exploration-exploitation tradeoff constant.

The **MDP_Lookahead** function repeatedly calls the UCT_Lookahead to select the next action and execute the selected action (using the step() method) until a goal state is reached:

<img src="images/mdp_lookahead.png" width="300">
   
Finally, we will need some **helper Functions** that depend on the environment to which we apply the UCT algorithm:
   - `V0`: Provides a heuristic value for terminal states.
   - `Sample`: Simulates a transition to a new state given a state-action pair.

Notice the difference between the step() method and the Sample() method: the step method executes the action in the world where the agent is operating while the Sample() method simulates the execution of the action to select the next best action through the UCT Rollout process. In this exercise, we use a copy of the environments where the agent operates to simulate the execution, hence the UCT rollout will be very accurate and the process very efficient. In real-life scenarios, we typically do not have such an accurate simulator/model of the environment.

In [ ]:
def Sample(sim_env, s, a):
    sim_env.state = s
    next_state, _, _, _ = sim_env.step(a)
    return next_state
def V0(s):
    return 0
def Select(sim_env, s, Q, N):
    if N[s] == 0:
        import random
        return random.choice(sim_env.Applicable(s))

    c = math.sqrt(2)
    
    # We use N[s] inside the log, which is now guaranteed to be > 0
    return max(sim_env.Applicable(s), key=lambda a: Q[s].get(a, 0) + c * math.sqrt(math.log(N[s]) / (1 + N.get((s, a), 0))))

In [15]:
def UCT_rollout(s, h, sim_env, Envelope=None, Q=None, N=None, metrics=None):
    """
    UCT-Rollout implementation.
    
    Parameters:
        s: Current state
        h: Current depth
        sim_env: Environment 
        Envelope: Set to store visited states
        Q: Dictionary for Q-values
        N: Dictionary for visit counts
        metrics: Dictionary to track evaluation metrics
    """
    # Initialize Envelope, Q, and N if not provided
    if Envelope is None:
        Envelope = set()
    if Q is None:
        Q = {}
    if N is None:
        N = {}

    # Base cases
    if s in sim_env.S_g():
        return 0
    if h == 0:
        return V0(s)
    
    if s not in Envelope:
        Envelope.add(s)
        N[s] = 0
        Q[s] = {}
        for a in sim_env.Applicable(s):
            Q[s][a] = 0
            N[(s,a)] = 0

    a = Select(sim_env, s, Q, N)
    s_next = Sample(sim_env, s, a)
    cost_rollout = sim_env.get_cost(s, a, s_next) + UCT_rollout(s_next, h-1, sim_env, Envelope, Q, N, metrics)
    Q[s][a] = (Q[s].get(a, 0) * N.get((s,a), 0) + cost_rollout) / (N.get((s,a), 0) + 1)
    N[s] += 1
    N[(s,a)] += 1

    return cost_rollout

In [16]:
import copy
import time
def UCT_Lookahead(s, h, n_sim, env, metrics):
    """
    UCT algorithm implementation.

    Parameters:
        s: Initial state
        h: Depth limit
        n_sim: The number of simulations to perform, this represents the termination condition
        env: environment in which to perform the procedure
        metrics: dictionary of evaluation metrics
    """
    if len(env.Applicable(s)) != 0:
        sim_env = copy.deepcopy(env)
        Q = {}
        N = {}
        Envelope = set()
        start_time = time.time()
        for _ in range(n_sim):
            metrics["total_iterations"] += 1
            UCT_rollout(s, h, sim_env, Envelope, Q, N, metrics)
        metrics["total_time"] += time.time() - start_time
        if s in Q:
            best_action = max(Q[s], key=Q[s].get)
            return best_action
    else:
        print("Dead end state reached")
        return None

In [17]:
def MDP_Lookahead(env, depth, simulations):

    eval_metrics = {
        "total_iterations": 0,
        "total_time": 0,
        "new_action_count": 0,
        "tried_action_count": 0
    }
    
    state = env.reset()          
    done = False
    while not done:
        action = UCT_Lookahead(state, depth, simulations, env, eval_metrics)
        print(UCT_log(env=env, state=state, action=action))
        state, _, done, _ = env.step(action)
        if done:
            print("Episode terminated in state ", state)

    return state, eval_metrics

In [18]:
env = gym.make("ProbabilisticPlanningDomain-v0")
state, metrics = MDP_Lookahead(env, 50, 50)
if state == env.goal_state:    
    print("\nGoal state reached!")

print("\n--- Evaluation Metrics ---")    
print(f"Total Iterations: {metrics['total_iterations']}")
print(f"Total Time (seconds): {metrics['total_time']:.4f}")
print(f"New Action Count: {metrics['new_action_count']}")
print(f"Tried Action Count: {metrics['tried_action_count']}")

Executed action m14 in state d1
Executed action m14 in state d1
Episode terminated in state  d4

Goal state reached!

--- Evaluation Metrics ---
Total Iterations: 100
Total Time (seconds): 0.0009
New Action Count: 0
Tried Action Count: 0


## Lava environments
You can test the UCT algorithm you implemented on the LavaFloor environment (visible in the figure) and its variations.

![Lava](images/lava.png)

The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls the agent can't cross, the floor is covered with lava, and there is a black pit of death.

Moreover, the agent can't comfortably perform its actions that instead have a stochastic outcome (visible in the figure):

![Dynact](images/dynact.png)

The available actions are Left, Right, Up and Down, denoted in the environment as "L", "R", "U", "D".
The action dynamics is the following:
- $P(0.8)$ of moving **in the desired direction**
- $P(0.1)$ of moving in a direction 90° with respect to the desired direction

Finally, since the floor is covered in lava, the agent receives a negative reward for each of its steps!

- -0.04 for each lava cell (L)
- -5 for the black pit (P). End of episode
- +1 for the treasure (G). End of episode


### Environment Properties 
In addition to the variables available in the previous environment, there are also a few more:

- $T$: matrix of the transition function $T(s, a, s') \rightarrow [0, 1]$
- $RS$: matrix of the reward function $R(s) \rightarrow \mathbb{R}$


#### Code Hints:

In [19]:
env_name = "LavaFloor-v0"
env = gym.make(env_name)
print("ENVIRONMENT:\n")
env.render()
# states are internally expressed using a numerical index, but you can convert coordinates to indexes:
current_state = env.pos_to_state(0, 0) 
next_state = env.pos_to_state(0, 1)
goal_state = env.pos_to_state(2, 3)

print("\nNumber of actions: ", env.action_space.n)
print("Actions: ", env.actions)
print("Reward of starting state:", env.RS[current_state])
print("Reward of goal state:", env.RS[goal_state])
print("Probability from (0, 0) to (1, 0) with action left:", env.T[current_state, 1, next_state])
print("Probability from (0, 0) to (2, 3) with action left:", env.T[current_state, 1, goal_state])

c=0
state_right = env.pos_to_state(0, 1) #state to the tight of start state
for i in range(1,101):
    current_state = env.pos_to_state(0, 0)
    state = env.sample(current_state, 1) #trying to go right
    if (state==state_right): 
        c+=1 #counting how many times the agent reaches the state to the right
        
#computing percentage of time agent reached the state to the right going right, should be around 80%           
print("\nPercentage of time agent reaches the state to the right:", c/i*100) 

print("\nTransition model for", env_name, ":") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("\nProbability of reaching", env.state_to_pos(next_state), "from", env.state_to_pos(state), "executing action", env.actions[i], ":", env.T[state, i, next_state])
print("\nReward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state:", env.state_to_pos(state) ,"(state type: ", env.grid[state],"):",env.RS[state])

ENVIRONMENT:

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]

Number of actions:  4
Actions:  {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
Reward of starting state: -0.04
Reward of goal state: 1.0
Probability from (0, 0) to (1, 0) with action left: 0.8
Probability from (0, 0) to (2, 3) with action left: 0.0

Percentage of time agent reaches the state to the right: 76.0

Transition model for LavaFloor-v0 :

Probability of reaching (0, 1) from (0, 0) executing action L : 0.0

Probability of reaching (0, 1) from (0, 0) executing action R : 0.8

Probability of reaching (0, 1) from (0, 0) executing action U : 0.1

Probability of reaching (0, 1) from (0, 0) executing action D : 0.1

Reward for non terminal states:  -0.04
Reward for state: (1, 3) (state type:  P ): -5.0
Reward for state: (2, 3) (state type:  G ): 1.0


#### In addition to the basic LavaFloor environment, you can also test some different environment versions, with different maps and rewards: *BiggerLavaFloor* and *VeryBadLavaFloor*. 

In [20]:
env = gym.make("LavaFloor-v0") # You can also test BiggerLavaFloor-v0 and VeryBadLavaFloor-v0 
print("ENVIRONMENT:\n")
env.render()
print("\n\n")

state, metrics = MDP_Lookahead(env, 20, 50) # You might want to change depth and simulation number if testing more complex environments
if state == env.goal_state:    
    print("\nGoal state reached!")  
else:
    print("\nThe agent fell in a pit!")

print("\n--- Evaluation Metrics ---")    
print(f"Total Iterations: {metrics['total_iterations']}")
print(f"Total Time (seconds): {metrics['total_time']:.4f}")
print(f"New Action Count: {metrics['new_action_count']}")
print(f"Tried Action Count: {metrics['tried_action_count']}")

ENVIRONMENT:

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]



Executed action U in state (0, 0)
Executed action U in state (0, 0)
Executed action U in state (0, 0)
Executed action U in state (0, 0)
Executed action D in state (0, 0)
Executed action D in state (1, 0)
Executed action D in state (1, 0)
Executed action U in state (1, 0)
Executed action U in state (0, 0)
Executed action L in state (0, 0)
Executed action L in state (0, 0)
Executed action D in state (1, 0)
Executed action D in state (2, 0)
Executed action D in state (2, 0)
Executed action R in state (2, 0)
Executed action R in state (2, 1)
Executed action R in state (2, 2)
Episode terminated in state  11

Goal state reached!

--- Evaluation Metrics ---
Total Iterations: 850
Total Time (seconds): 0.4379
New Action Count: 0
Tried Action Count: 0
